In [1]:
from drn_interactions.load import load_neurons, load_spikes, get_derived_data_dir
from drn_interactions.stats import cv_isi_burst, is_burster, median_burst_interval
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

In [2]:
outdir = get_derived_data_dir() / "neurons"
outdir.mkdir(exist_ok=True)

In [3]:
neurons = load_neurons()
df_spikes = load_spikes("pre")

In [4]:
cv2_burst = df_spikes.groupby("neuron_id")["spiketimes"].apply(cv_isi_burst, thresh=0.05)
is_burst = df_spikes.groupby("neuron_id")["spiketimes"].apply(is_burster, burst_thresh=0.05, frac_bursts=0.1)
med_burst = df_spikes.groupby("neuron_id")["spiketimes"].apply(median_burst_interval, thresh=0.05)



df = (
    neurons
    .merge(cv2_burst.to_frame(name="cv2_burst").reset_index())
    .merge(is_burst.to_frame(name="is_burster").reset_index())
    .merge(med_burst.to_frame(name="median_burst").reset_index())
)
df.head(2)


c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


,neuron_id,waveform_width,peak_asymmetry,mean_firing_rate,cv2_isi,recording_session_id,cluster_id,channel,is_single_unit,session_name,group_name,experiment_name,cv2_burst,is_burster,median_burst
0,1,16.0,0.065574,0.601693,0.467315,3,0,6,1,ESHOCK_03_LOC1,shock,ESHOCK,0.449766,True,0.041733
1,4,9.0,0.037344,2.105512,0.824797,3,3,17,1,ESHOCK_03_LOC1,shock,ESHOCK,0.638198,True,0.022283


In [5]:
df.to_csv(outdir / "burst_features.csv", index=False)